In [1]:
import torch
import os
import cv2
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [2]:
from torchvision import transforms

In [3]:
transform_data = transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.RandomVerticalFlip(),
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop((112)),
        transforms.ToTensor(),
    ]
)

In [4]:
def load_data(img_size=112):
    data = []
    labels = {}
    index = -1
    for label in os.listdir('./data/'):
        index += 1
        labels[label] = index
    print(len(labels))
    X = []
    y = []
    for label in labels:
        for file in os.listdir(f'./data/{label}/'):
            path = f'./data/{label}/{file}'
            img = cv2.imread(path)
            img = cv2.resize(img,(img_size,img_size))
            data.append([np.array(transform_data(np.array(img))),labels[label]])
            X.append(np.array(transform_data(np.array(img))))
            y.append(labels[label])
    np.random.shuffle(data)
    np.save('./data.npy',data)
    VAL_SPLIT = 0.25
    VAL_SPLIT = len(X)*VAL_SPLIT
    VAL_SPLIT = int(VAL_SPLIT)
    X_train = X[:-VAL_SPLIT]
    y_train = y[:-VAL_SPLIT]
    X_test = X[-VAL_SPLIT:]
    y_test = y[-VAL_SPLIT:]
    X = torch.from_numpy(np.array(X))
    y = torch.from_numpy(np.array(y))
    X_train = np.array(X_train)
    X_test = np.array(X_test)
    y_train = np.array(y_train)
    y_test = np.array(y_test)
    X_train = torch.from_numpy(X_train)
    X_test = torch.from_numpy(X_test)
    y_train = torch.from_numpy(y_train)
    y_test = torch.from_numpy(y_test)
    return X,y,X_train,X_test,y_train,y_test

In [5]:
X,y,X_train,X_test,y_train,y_test = load_data()

50


/home/indika/anaconda3/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


## Modelling

In [6]:
import torch.nn as nn
import torch.nn.functional as F

In [7]:
class BaseLine(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,32,5)
        self.conv2 = nn.Conv2d(32,64,5)
        self.conv2batchnorm = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64,128,5)
        self.fc1 = nn.Linear(128*10*10,256)
        self.fc2 = nn.Linear(256,128)
        self.fc3 = nn.Linear(128,50)
        self.relu = nn.ReLU()
    
    def forward(self,X):
        preds = F.max_pool2d(self.relu(self.conv1(X)))
        preds = F.max_pool2d(self.relu(self.conv2batchnorm(self.conv2(preds))))
        preds = F.max_pool2d(self.relu(self.conv3(preds)))
        preds = preds.view(-1,128*10*10)
        preds = self.relu(self.fc1(preds))
        preds = self.relu(self.fc2(preds))
        preds = self.relu(self.fc3(preds))
        return preds

In [8]:
device = torch.device('cuda')

In [9]:
model = BaseLine().to(device)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=1e-03)

In [10]:
PROJECT_NAME = 'Car-Brands-Images-Clf'

In [11]:
import wandb

In [12]:
EPOCHS = 100
BATCH_SIZE = 32

In [ ]:
wandb.init(project=PROJECT_NAME,name='custom-model-baseline')
f